In [1]:
PATH_TO_SAMPLE_FILE = "/home/chendian/CDConfusor/exp/data/cn/Wang271k/dcn_train.tsv"
lines = [line for line in open(PATH_TO_SAMPLE_FILE, 'r')]
len(lines)

272099

In [8]:
from tqdm import tqdm
from collections import Counter

char_level_pairs = []

for line in tqdm(lines):
    err, cor = line.rstrip().split('\t')[:2]
    err = err.replace(' ', '')
    cor = cor.replace(' ', '')
    if err == cor:
        continue
    else:
        faulty_position = []
        for i, (_e, _c) in enumerate(zip(err, cor)):
            if _e != _c:
                char_level_pairs.append((_e, _c))


ct = Counter(char_level_pairs).most_common()

100%|██████████| 281381/281381 [00:04<00:00, 60302.75it/s]


## char-level confusion

In [12]:
import jieba
import random
from tqdm import tqdm
from copy import deepcopy

# char-level confusion from SpellGCN
cfs_path = '../data/spellGraphs.txt'
char_cfs = {}
for line in open(cfs_path, 'r'):
    l, r, t = line.strip().split('|')
    if t in ['同音同调', '同音异调', '近音异调', '近音同调']:
        t = '近音'
    char_cfs.setdefault(t, {})
    char_cfs[t].setdefault(l, [])
    char_cfs[t][l].append(r)
backup_cfs = deepcopy(char_cfs)
print("char-cfs Loaded", list(char_cfs.keys()))


def char_confusor(char):
    # always take different token
    take = char
    candidates = char_cfs['近音'].get(char, [char])
    if candidates:
        take = random.choice(candidates)
        if take != char:
            char_cfs['近音'][char].remove(take)
    else:
        if backup_cfs['近音'][char]:
            char_cfs['近音'][char] = [_c for _c in backup_cfs['近音'][char]]
    return take


def augment_single_sample(err, cor, confusor):
    faulty_position = []
    for i, (_e, _c) in enumerate(zip(err, cor)):
        if _e != _c:
            faulty_position.append((i, _e, _c))
    for i, e, c in faulty_position:
        assert cor[i] == c
        cor = f"{cor[:i]}{confusor(c)}{cor[i+1:]}"
    return cor

dir_path = '../exp/data/cn/'
# SIGHAN
# src_path = dir_path + 'sighan15/sighan15_train.tsv'
# tgt_path = dir_path + 'sighan15/sighan15_train.augc.tsv'

# Wang271K + SIGHAN
src_path = dir_path + 'Wang271k/dcn_train.tsv'
tgt_path = dir_path + 'Wang271k/dcn_train.augc.tsv'
with open(tgt_path, 'w') as f:
    for line in tqdm(open(src_path, 'r')):
        err, cor = line.strip().split('\t')
        aug_err = augment_single_sample(err, cor, confusor=char_confusor)
        if len(err) == len(cor):
            f.write(f"{aug_err}\t{cor}\n")


char-cfs Loaded ['形近', '近音', '同部首同笔画']


272099it [00:03, 72901.51it/s]


## word-level confusion
> version 1.0

In [1]:
# import json
# ime = json.load(open('../data/input_candidates.google.json', 'r'))

In [1]:
import sys
sys.path.append('../')

import json
import jieba
import random
from tqdm import tqdm

from pypinyin import lazy_pinyin

mapping = {}
used_conf = {}

"""
from src.confusor import Confusor as ConfusorV1
conf = ConfusorV1(
    cand_pinyin_num=10, 
    cos_threshold=(0., .75), 
    method='all-similar single-freedom', 
    token_sample_mode='sort', 
    pinyin_sample_mode='sort',  # special
    weight=[1., 0, .2],   # pinyin score, similarity score, word freq score, IME ranking
    conf_size=300, ime_weight=1,
    debug=False)
conf.conf_with_scores = True
"""

from src.confusor_v2 import Confusor as ConfusorV2
cfs = ConfusorV2()

Loaded 364 items from /home/chendian/CDConfusor//data/red_cache.pkl
Loaded 186258 items from /home/chendian/CDConfusor//data/cfs_cache.pkl
Loading is_memory.json (153.94MB) cost 5.083 seconds.
Loaded 178481 items from /home/chendian/CDConfusor//data/is_memory.json
Loading ime_memory.json (4647.33MB) cost 432.553 seconds.
Loaded 190194 items from /home/chendian/CDConfusor//data/ime_memory.json


In [ ]:
for item in cfs.confusor_cache['jinji'].values():
    for k, v in item:
        print(k)

{2: [('紧急', '0'),
  ('禁忌', '1'),
  ('晋级', '2'),
  ('谨记', '3'),
  ('金鸡', '4'),
  ('近几', '5'),
  ('近畿', '6'),
  ('进即', '7-0'),
  ('进急', '7-1'),
  ('进几', '7-2'),
  ('进及', '7-3'),
  ('进机', '7-4'),
  ('进级', '7-5'),
  ('进记', '7-6'),
  ('进集', '7-7'),
  ('进既', '7-8'),
  ('进鸡', '7-9'),
  ('进寄', '7-10'),
  ('进极', '7-11'),
  ('进吉', '7-12'),
  ('进己', '7-13'),
  ('进基', '7-14'),
  ('进挤', '7-15'),
  ('进季', '7-16'),
  ('进计', '7-17'),
  ('进剂', '7-18'),
  ('进继', '7-19'),
  ('金即', '8-0'),
  ('金急', '8-1'),
  ('金几', '8-2'),
  ('金及', '8-3'),
  ('金机', '8-4'),
  ('金级', '8-5'),
  ('金记', '8-6'),
  ('金集', '8-7'),
  ('金既', '8-8'),
  ('金鸡', '8-9'),
  ('金寄', '8-10'),
  ('金极', '8-11'),
  ('金吉', '8-12'),
  ('金己', '8-13'),
  ('金基', '8-14'),
  ('金挤', '8-15'),
  ('金季', '8-16'),
  ('金计', '8-17'),
  ('金剂', '8-18'),
  ('金继', '8-19'),
  ('近即', '9-0'),
  ('近急', '9-1'),
  ('近几', '9-2'),
  ('近及', '9-3'),
  ('近机', '9-4'),
  ('近级', '9-5'),
  ('近记', '9-6'),
  ('近集', '9-7'),
  ('近既', '9-8'),
  ('近鸡', '9-9'),
  ('近寄', '9-10'),
  ('

In [9]:
[k # (k, round(v, 3)) 
 for k, v in cfs('紧急', return_score=True)]

['紧急',
 '解决',
 '仅仅',
 '禁忌',
 '经济',
 '晋级',
 '积极',
 '紧紧',
 '谨记',
 '姐姐',
 '金鸡',
 '进军',
 '近几',
 '拒绝',
 '锦江',
 '近畿',
 '接近',
 '进即',
 '金价',
 '进急',
 '金即',
 '究竟',
 '晋江',
 '进几',
 '金急',
 '近即',
 '基金',
 '进价']

In [ ]:
def word_confusor(word, random_select=True):
    if word not in used_conf or len(used_conf.get(word, [])) == 0:
        used_conf[word] = [] 
        res = conf(word)
        min_score = min([x[1] for x in res])
        for w, score in res:
            used_conf[word].extend([w] * int((score - min_score) // 0.01 + 1))
        mapping[word] = [w for w in used_conf[word]]
    """
    elif len(used_conf[word]) == 0:  # all out
        if word in mapping:
            used_conf[word] = mapping[word]
    """
    if word in used_conf[word]:
        used_conf[word].remove(word)
    if random_select:
        ret = random.choice(used_conf[word])
        used_conf[word].remove(ret)
    else:
        ret = used_conf[word][0]
        used_conf[word] = used_conf[word][1:]
    return ret


def word_confusor_ime(word, random_select=True):
    py_case = lazy_pinyin(word)
    complete_input_sequence = ''.join(py_case)
    if word not in used_conf or len(used_conf.get(word, [])) == 0:
        used_conf[word] = [] 
        res = []
        for omission in range(1, len(py_case[-1])):
            inp_seq = complete_input_sequence[:-omission]
            if inp_seq not in ime:
                print(f"{inp_seq} not in IME Records.")
            candidates = ime[inp_seq]
            res.extend([(c, 1 / (idx + 1)) for idx, c in enumerate(candidates) if len(c) <= len(word)])
        for w, score in res:
            used_conf[word].extend([w] * int(score // 0.01 + 1))
        if len(used_conf[word]) == 0:
            return word_confusor(word, random_select=random_select)
    if word in used_conf[word]:
        used_conf[word].remove(word)
    if random_select:
        ret = random.choice(used_conf[word])
        used_conf[word].remove(ret)
        if len(ret) < len(word):
            return ret + word_confusor_ime(
                word[len(ret):], random_select=random_select)
    else:
        ret = used_conf[word][0]
        used_conf[word] = used_conf[word][1:]
    return ret

In [14]:

def word_confusor_cfs(word, random_select=True, ignore_word=None):
    if word not in used_conf or len(used_conf.get(word, [])) == 0:
        used_conf[word] = [] 
        res = []
        
        candidates = cfs(word, return_score=True)
        # print(candidates)
        res.extend(
            [(c, score) for idx, (c, score) in enumerate(candidates) 
             if len(c) == len(word)])

        for w, score in res:
            used_conf[word].extend([w] * int( (1. - score) ** 2 // 0.01 + 1))
    
    if word in used_conf[word]:
        used_conf[word].remove(word)
    if ignore_word is not None:
        if ignore_word in used_conf[word]:
            used_conf[word].remove(ignore_word)

    if len(used_conf[word]) == 0:
        try:  # sample single char in it.
            char = random.choice(word)
            _c = word_confusor_cfs(char, random_select=True)
            ret = ''.join([_c if c == char else c for c in word])
            return ret
        except Exception as e:
            print(e)
            return word

    if random_select:
        ret = random.choice(used_conf[word])
        used_conf[word].remove(ret)
    else:
        ret = used_conf[word][0]
        used_conf[word] = used_conf[word][1:]

    return ret


def augment_single_sample(err, cor, confusor):
    faulty_position = []
    for i, (_e, _c) in enumerate(zip(err, cor)):
        if _e != _c:
            faulty_position.append((i, _e, _c))
    es, cs, streak = "", "", []
    for i, e, c in faulty_position:
        assert cor[i] == c
        if len(streak) == 0:
            es, cs, streak = e, c, [i]
        elif i == streak[-1]+1:
            es += e
            cs += c
            streak.append(i)
        elif i != streak[-1] + 1 and len(cs) > 0:
            cor = f"{cor[:streak[0]]}{confusor(cs)}{cor[streak[-1]+1:]}"
            es, cs, streak = e, c, [i]
    else:
        if len(cs) > 0:
            cor = f"{cor[:streak[0]]}{confusor(cs)}{cor[streak[-1]+1:]}"
            es, cs, streak = "", "", []
    return cor

import Pinyin2Hanzi 
def augment_single_sample_jieba(err, cor, confusor):
    faulty_position = []
    words = jieba.lcut(err)
    pivot = 0
    for i, w in enumerate(words):
        _e = err[pivot: pivot+len(w)]
        _c = cor[pivot: pivot+len(w)]
        if Pinyin2Hanzi.is_chinese(_e) and _e != _c:
            if len(_e) > 2:
                for offset, (__e, __c) in enumerate(zip(_e, _c)):
                    if __e != __c:
                        faulty_position.append(
                            (pivot+offset, pivot+offset+1, __e, __c))
            else:
                faulty_position.append((pivot, pivot+len(w), _e, _c))
        pivot += len(w)
    for i, j, _, c in faulty_position:
        try:
            assert c == cor[i:j]
            # not the same with current error
            changed_word = confusor(c, ignore_word=err[i:j])  
            cor = f"{cor[:i]}{changed_word}{cor[j:]}"
        except Exception as e:
            print(e)
            print(cor, cor[i:j])
    return cor


same, faulty = 0, 0
src_path = '../exp/data/tmp/findoc_train.230329.augw2.tsv'
tgt_path = '../exp/data/tmp/findoc_train.230329.augw3.tsv'


with open(tgt_path, 'w') as f:
    for idx, line in tqdm(enumerate(open(src_path, 'r'))):
        err, cor = line.strip().split('\t')
        # aug_err = augment_single_sample(err, cor, confusor=word_confusor)
        aug_err = augment_single_sample_jieba(
            err, cor, confusor=word_confusor_cfs)
        if len(aug_err) == len(cor):
            f.write(f"{aug_err}\t{cor}\n")
            if aug_err == cor:
                same += 1
            else:
                f.write(f"{cor}\t{cor}\n")
                faulty += 1
        else:
            print(len(aug_err), aug_err)
            print(len(cor), cor)
            print(same, faulty)

same, faulty

In [ ]:
cfs.ism.ime_memory['hutao']

In [11]:
cfs.ism.save_memory()
cfs.ism.update_memory_from_tmp()

In [3]:
!ls -lht /data/chendian/CDConfusor/tmp/

total 804M
-rw-rw-r-- 1 chendian chendian 650M Mar 23 23:41 ime_memory.json
-rw-rw-r-- 1 chendian chendian 154M Mar 22 19:05 memory.json


In [2]:
import jieba
jieba.lcut("--27632常被用来进行稀释血管中血液的实验，可能有助于研发高血压治疗剂。")

['--',
 '27632',
 '常',
 '被',
 '用来',
 '进行',
 '稀释',
 '血管',
 '中',
 '血液',
 '的',
 '实验',
 '，',
 '可能',
 '有助于',
 '研发',
 '高血压',
 '治疗剂',
 '。']

In [3]:
same, faulty

(158432, 113667)

## Word Confusion
> Version 2.0